#### <u><font color="orange">**Notes:**</font></u>


<p>In this notebook, I will attempt to complete Exercise 2. It is intended to be readable by my teammates.</p> <br>

## <u>Exercise 2:</u>

<ol>
    <li>Implement RLS and (1+1) EA introduced in w3</li>
    <li>Run Random Search, RLS and (1+1) EA 10 times on the problems below.</li>
</ol>


## <u> Problems </u>

$$
\begin{array}{ll}
\text{F1 OM: } & \{0,1\}^n \to [0..n], x \mapsto \sum_{i=1}^{n}x_i \\
\text{F3 LO: } & \{0,1\}^n \to [0..n], x \mapsto \max{i \in [0..n] | \forall j\leq i: x_j = 1} = \sum_{i=1}^{n}\prod_{j=1}^{i} x_j \\
\text{LHW: } & f:\{0,1\}^n\to \R, x \mapsto \sum_{i}ix_i \\
\text{LABS: } & x\mapsto \frac{n^2}{2\sum_{k=1}^{n-1}(\sum_{i=1}^{n-k}s_i s_{i+k})^2}, \text{where } s_i = 2x_i - 1. \\
\text{N-Queens Problem} & \\
\text{Concentrated Trap} & \\
\text{NK Landscapes (NKL)} &
\end{array}
$$


### **<u>Terminologies and Definitions</u>**


- `n` (int) := synonymous with the dimension of the problem or the problem size.
- `budget` (int) := (int) or the number of iterations or function evalutations. 

In [1]:
import os 
import numpy as np 
from shutil import rmtree 
import glob 

In [2]:
def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)


def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)


def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())


clean()

In [3]:
import ioh 
from ioh import ProblemClass, logger 

In [4]:
#  a list of problem can be accessed via the base classes 
real_problems: dict[int, str] = ioh.problem.RealSingleObjective.problems 
print(real_problems)

{1: 'Sphere', 2: 'Ellipsoid', 3: 'Rastrigin', 4: 'BuecheRastrigin', 5: 'LinearSlope', 6: 'AttractiveSector', 7: 'StepEllipsoid', 8: 'Rosenbrock', 9: 'RosenbrockRotated', 10: 'EllipsoidRotated', 11: 'Discus', 12: 'BentCigar', 13: 'SharpRidge', 14: 'DifferentPowers', 15: 'RastriginRotated', 16: 'Weierstrass', 17: 'Schaffers10', 18: 'Schaffers1000', 19: 'GriewankRosenbrock', 20: 'Schwefel', 21: 'Gallagher101', 22: 'Gallagher21', 23: 'Katsuura', 24: 'LunacekBiRastrigin', 30: 'UniformStarDiscrepancy10', 31: 'UniformStarDiscrepancy25', 32: 'UniformStarDiscrepancy50', 33: 'UniformStarDiscrepancy100', 34: 'UniformStarDiscrepancy150', 35: 'UniformStarDiscrepancy200', 36: 'UniformStarDiscrepancy250', 37: 'UniformStarDiscrepancy500', 38: 'UniformStarDiscrepancy750', 39: 'UniformStarDiscrepancy1000', 40: 'SobolStarDiscrepancy10', 41: 'SobolStarDiscrepancy25', 42: 'SobolStarDiscrepancy50', 43: 'SobolStarDiscrepancy100', 44: 'SobolStarDiscrepancy150', 45: 'SobolStarDiscrepancy200', 46: 'SobolStarDis

In [5]:
def compute_mean(X: np.ndarray) -> float:
    return float(np.mean(X))


def compute_sd(X: np.ndarray) -> float:
    return float(np.std(X))

In [6]:
def run_experiment(problem: ioh.problem.PBO, algorithm, num_runs = 10):
    """
    Run an experiment of a given algorithm on a given problem for a number of runs (budget)
    - `problem`: an ioh problem instance
    - `algorithm`: a callable that takes a problem as input and runs the algorithm on it
    - `num_runs`: number of independent runs to perform
    """
    # For a known problem 18 w/ 32 variables, we know optimum = 8
    if problem.meta_data.problem_id == 18 and problem.meta_data.n_variables == 32:
        optimum = 8
    else:
        optimum = problem.optimum.y # otherwise, calculate optimum???
    print(optimum)
    # =============================================================


    for run in range(num_runs):
        # run the algorithm on the problem 
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found: {problem.state.current_best.y: .3f}")

        # reset the problem 
        problem.reset()

### <u>Let's begin implementing our algorithms</u>

In [37]:
# implement randomized local search 
class RandomizedLocalSearch:
    def __init__(self, budget: int):
        self.budget = budget

    def __call__(self, problem: ioh.problem.PBO) -> None:
        # initialize a random solution 
        current = np.random.randint(0, 2, size=problem.meta_data.n_variables)
        current_value = problem(current.tolist())
        evaluations = 1

        while evaluations < self.budget:
            # create a neighbor by flipping one random bit
            neighbor = current.copy()
            flip_index = np.random.randint(0, problem.meta_data.n_variables)
            neighbor[flip_index] = 1 - neighbor[flip_index] # flip the bit
            neighbor_value = problem(neighbor)
            evaluations += 1

            # if the neighbor is better or equal, move to the neighbor
            if neighbor_value >= current_value:
                current, current_value = neighbor, neighbor_value

In [38]:
# implement (1+1) EA
class OnePlusOneEA:
    def __init__(self, budget: int):
        self.budget = budget

    def __call__(self, problem: ioh.problem.PBO) -> None:
        # initialize a random solution 
        current = np.random.randint(0, 2, size=problem.meta_data.n_variables)
        current_value = problem(current.tolist())
        evaluations = 1

        while evaluations < self.budget:
            # create an offspring by flipping each bit with probability 1/n
            offspring = current.copy()
            for i in range(problem.meta_data.n_variables):
                if np.random.rand() < 1 / problem.meta_data.n_variables:
                    offspring[i] = 1 - offspring[i] # flip the bit
            offspring_value = problem(offspring)
            evaluations += 1

            # if the offspring is better or equal, move to the offspring
            if offspring_value >= current_value:
                current, current_value = offspring, offspring_value

In [39]:
# random search class 
class RandomSearch:
    def __init__(self, budget: int) -> None:
        """
        - `budget`: (int) number of function evaluations to perform
        """
        self.budget = budget
    
    def __call__(self, problem_func: ioh.problem.PBO) -> None:
        """
        - `problem`: an ioh problem instance
        """
        for _ in range(self.budget):
            # sample a random solution 
            X: np.ndarray = np.random.randint(2, size=problem_func.meta_data.n_variables) # randomized binary solution: a vector of 0s and 1s of length n_variables 
            # evaluate the solution 
            problem_func(X.tolist()) # unfortunately, ioh does not accept numpy arrays as input so we need to convert it to a list


#### Gather the required problems 

In [8]:
n = problem_size = 100 

In [9]:
p1 = ioh.get_problem(fid=1, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # Sphere
# grab problem 2 
p2 = ioh.get_problem(fid=2, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # 
# grab problem 3 
p3 = ioh.get_problem(fid=3, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #
# grab problem 18 
p18 = ioh.get_problem(fid=18, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p23 = ioh.get_problem(fid=23, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p24 = ioh.get_problem(fid=24, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 24
p24 = ioh.get_problem(fid=24,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 25
p25 = ioh.get_problem(fid=25,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue]

# A list of the whole problems 
Problems: list[ioh.problem.PBO] = [p1, p2, p3, p18, p23, p24, p25]

In [67]:
# create a list of 3 loggers for each algorithm -- Random Search, RLS (random local search) and (1+1)-EA
root_folder = "my-experiment"  # pyright: ignore[reportArgumentType]
if os.path.exists(root_folder):
    rmtree(root_folder, ignore_errors=True)

folder_name: str = "run"
algorithm_names: list[str] = ["RandomSearch", "RLS", "OnePlusOneEA"]
# info should be a short description of the algorithm
algorithm_info = [
    "Random Search Algorithm",
    "Random Local Search Algorithm",
    "(1+1)-EA Algorithm"
]

# for name in algorithm_names:

In [68]:
budget = num_iterations = 100000 # budget is the number of function evaluations

In [69]:
num_experiments = 10 # number of independent runs

In [70]:
Loggers: list[logger.Analyzer] = []
# create a list of loggers for each algorithm
for i, name in enumerate(algorithm_names):
    l = logger.Analyzer(
            root=root_folder,  # pyright: ignore[reportArgumentType]
            folder_name=folder_name,
            algorithm_name=name,
            algorithm_info=algorithm_info[i]
        )
    Loggers.append(
       l
    )



In [63]:
L_random_search = Loggers[0]
L_RLS = Loggers[1]
L_OnePlusOneEA = Loggers[2]

#### testing randoms search on problem 1

In [ ]:
p1.attach_logger(L_random_search)
algorithm = RandomSearch(budget=budget)
run_experiment(problem=p1, 
            algorithm=algorithm, 
            num_runs=num_experiments)

#### Experiment with the <i>del</i> keyword

In [52]:
L_random_search = Loggers[0]

In [53]:
del L_random_search

In [54]:
Loggers[0]

<Analyzer my-experiment/run>

### Run random search on all our problems (fingers crossed!)

In [33]:
# we can define our random search algorithm once and reuse it thorughout each loop 
random_search_algorithm = RandomSearch(budget=budget)

In [ ]:
i = 0
for problem in Problems:
    print(f"================ Begin {i+1} =======================")
    L_random_search = Loggers[0]
    problem.attach_logger(L_random_search)
    # run random search
    run_experiment(problem=problem, 
                algorithm=random_search_algorithm, 
                num_runs=num_experiments)
    print(f"================ End {i + 1} =======================")
    del L_random_search # this is important to avoid logging issues ...
    i += 1

<p>An example of what would happen if we put our log outside instead ... </p>
<p>One can try to guess its behavior before uncommenting and running it.</p>

In [47]:
# i = 0
# L_random_search = Loggers[0]
# for problem in Problems:
#     print(f"================ Begin {i+1} =======================")
#     problem.attach_logger(L_random_search)
#     # run random search
#     run_experiment(problem=problem, 
#                 algorithm=random_search_algorithm, 
#                 num_runs=num_experiments)
#     print(f"================ End {i + 1} =======================")
#     del L_random_search # this is important to avoid logging issues ...
#     i += 1

### <u>Enumerate over all our 3 algorithms</u>

<p>It looks like it worked out pretty well. Now we can include the other 2 algorithms into the mix!</p>

In [48]:
# first, instantiate each algorithm 
rls_algorithm = RandomizedLocalSearch(budget=budget)
one_plus_one_ea_algorithm = OnePlusOneEA(budget=budget)
random_search_algorithm = RandomSearch(budget=budget)


In [50]:
Algorithms = [random_search_algorithm, rls_algorithm, one_plus_one_ea_algorithm]

In [71]:
i = 0
for algorithm in Algorithms:
    print(f"================ Begin Algorithm {i+1} =======================")
    for problem in Problems:
        print(f"========== Begin Problem {problem.meta_data.problem_id} =================")
        L = Loggers[i]
        problem.attach_logger(L)
        
        # run the algorithm
        run_experiment(problem=problem, 
                    algorithm=algorithm, 
                    num_runs=num_experiments)
        del L # this is important to avoid logging issues ...
        print(f"========== End Problem {problem.meta_data.problem_id} =================")
    print(f"=============== End Algorithm {i+1} =======================")
    i += 1

================ Begin Algorithm 1 =======================
========== Begin Problem 1 =================
100.0
run: 1 - best found:  72.000
run: 2 - best found:  70.000
run: 3 - best found:  70.000
run: 4 - best found:  71.000
run: 5 - best found:  72.000
run: 6 - best found:  74.000
run: 7 - best found:  70.000
run: 8 - best found:  69.000
run: 9 - best found:  72.000
run: 10 - best found:  71.000
========== End Problem 1 =================
========== Begin Problem 2 =================
100.0
run: 1 - best found:  18.000
run: 2 - best found:  16.000
run: 3 - best found:  21.000
run: 4 - best found:  15.000
run: 5 - best found:  16.000
run: 6 - best found:  17.000
run: 7 - best found:  17.000
run: 8 - best found:  18.000
run: 9 - best found:  19.000
run: 10 - best found:  19.000
========== End Problem 2 =================
========== Begin Problem 3 =================
5050.0
run: 1 - best found:  3834.000
run: 2 - best found:  3744.000
run: 3 - best found:  3753.000
run: 4 - best found:  3771